In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
data =datasets.load_iris()
import numpy as np
x, y = data.data, data.target
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=0)


class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
    def equil_distance(self,x1, x2):
        sum = 0
        for i in range(len(x1)):
            sum += (x1[i] - x2[i]) ** 2
        return np.sqrt(sum)
    def find_center(self,X):
        random_args = np.random.choice(len(X), 10)
        centroids = X[random_args]
        return np.array(centroids)
    
    def one_hot_repre(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr
    
    def interpolitation_matrix(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([1 / np.exp(-self.equil_distance(x, c) / s ** 2) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)

    def fit(self):

        self.centroids = self.find_center(self.X)

        dMax = np.max([self.equil_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
        self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.interpolitation_matrix(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.one_hot_repre(self.y, self.number_of_classes)
    def performace_score(self):
        
        RBF_list_tst = self.interpolitation_matrix(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

In [2]:

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=3,k=16)

RBF_CLASSIFIER.fit()
RBF_CLASSIFIER.performace_score()

Accuracy:  0.9333333333333333
